In [7]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
import argparse
from datetime import datetime
import pytz
import csv

c:\Users\Tardy\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject


In [ ]:
def get_scale(enc):
    s = enc.layers[0].output_shape[0][1]
    s = 1024//s
    return s

In [ ]:
def change_model(dec,s):
    inp_shape = (32//s,32//s,8)
    code_input_layer = keras.Input(shape=inp_shape, name="CODE_INPUT")
    x = code_input_layer
    for lay in dec.layers[1:]:
        x = lay(x)
    new_dec = keras.Model(code_input_layer,x)
    return new_dec

In [ ]:
fld = f"./autoencoder_models/2022-08-22-11-06"
encoder = keras.models.load_model(f"{fld}/encoder.h5", compile=False)
decoder = keras.models.load_model(f"{fld}/decoder.h5", compile=False)
encoder.summary()

In [ ]:
decoder.layers[-1].output_shape[1]

In [ ]:
s = get_scale(encoder)
d = change_model(decoder,s)
d.summary()

In [1]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import os

In [25]:
def timegan_sat(data,fld,n,s):
    data = data*255.0
    # dir_path = f"./timeGAN_results/sat/{fld}"
    # if not os.path.exists(f"{dir_path}"):
    #     os.mkdir(f"{dir_path}")
    
    bckg = Image.new(mode="RGB",size=(1270,220),color=(255,255,255))
    for k in range(n):
        if k%6 == 0:
            bckg.save(f"./temp/{k//6}.png")
            bckg = Image.new(mode="RGB",size=(1270,220),color=(255,255,255))
            x = 10
        img = data[k].reshape((data.shape[1],data.shape[1]))
        img = Image.fromarray(img)
        bckg.paste(img.resize((200,200)),(x,10))
        x += 210

        

In [4]:
from data_processing import import_sat

In [29]:
n = 120
data = import_sat("./data/sat_images/",n,4)
fld = "./autoencoder_models/2022-08-22-12-21"
encoder = keras.models.load_model(f"{fld}/encoder.h5", compile=False)
decoder = keras.models.load_model(f"{fld}/decoder.h5", compile=False)
enc_data = encoder.predict(data)
dec_data = decoder.predict(enc_data)
timegan_sat(dec_data,"",n,6)
print(data.shape,enc_data.shape,dec_data.shape)

4/4 [==============================] - 3s 620ms/step
(120, 256, 256, 1) (120, 8, 8, 8) (120, 256, 256, 1)


In [17]:
dec_data.shape[1]

256